In [ ]:
#Script que insere dados no banco de dados usando o maapeamento da tabela com Metadata e a abertura de uma
#Sessao, serve para operacoes mais complexas como UPDATE
import pandas as pd 
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, VARCHAR, text
from sqlalchemy.orm import sessionmaker
import pyodbc

server = "DESKTOP-HV85ADA\\PROJETO_MERCADO"
bancodados = "MERCADO_OLTP"
driver = "ODBC Driver 17 for SQL Server"
#user = ""
#senha = ""

conexaostring = f"mssql+pyodbc://{server}/{bancodados}?driver={driver}&Trusted_Connection=yes"
#conn_str = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver}"
engine = create_engine(conexaostring)

nome_tabela = "CFOP"

#Mapeando a tabela existente no banco de dados
metadata = MetaData()
tabela = Table(nome_tabela, metadata,
                                 Column("IDCODCFOP", Integer, primary_key=True, autoincrement=True),
                                 Column("COD_CFOP",  String(20)),
                                 Column("DESCRICAO", String(100))           
                                         
)

Session = sessionmaker(bind=engine)
session = Session()

lista_dados_cfop = {
       "COD_CFOP":          ["1102","2102","5101","5102","6101","6102"],
       "DESCRICAO":         ["COMPRAS DE MERCADORIA","COMPRAS DE MERCADORIA","VENDAS DE MERCADORIA",
                             "VENDAS DE MERCADORIA ADQ TERCEIROS","VENDAS DE MERCADORIA",
                             "VENDAS DE MERCADORIA ADQ TERCEIROS"]       

}

df_CFOP = pd.DataFrame(lista_dados_cfop)

#Usar paar nomear as colunas do dataframe com os mesmos nomes das colunas do banco de dados
#df_CFOP.rename(columns={'Coluna1': 'COD_CFOP', 'Coluna2': 'DESCRICAO'}, inplace=True)

tipo_dados = {

    "COD_CFOP":  VARCHAR(20),
    "DESCRICAO": VARCHAR(100)

}

try:
    with engine.connect() as conn:
        conn.execute(text(f"TRUNCATE TABLE {nome_tabela}"))  #limpa os dados existentes, pode ser removido do codigo
        conn.commit()

    df_CFOP.to_sql(
                name=nome_tabela,
                con=engine,
                if_exists="append", #se usar o replace ele faz um drop table da tabela no banco de dados e cria uma com a estrutura do dataframe
                index=False,
                dtype=tipo_dados,
)
    print("Dados inseridos com sucesso")
except Exception as e:
    print(f"Erro ao inserir dados: {e}")
finally:
    session.close()